In [34]:
from bs4 import BeautifulSoup
from __future__ import division, print_function
import tensorflow_hub as hub
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
import nltk 
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

In [2]:
data=pd.read_csv('train_data.csv',header=None)
data.columns=['Class','Message']

In [3]:
data.head()

,Class,Message
0,phishing,From acteamco@server2.aking.com.my Sat Jan 24...
1,phishing,From rm@transservis.az Wed Jan 21 03:30:32 20...
2,phishing,From MAILER-DAEMON Thu Sep 28 09:57:25 2017\n\...
3,phishing,From m2266@cscv.qc.ca Fri Feb 6 08:48:37 2015...
4,phishing,From roberta@analiselaboratorios.com.br Mon M...


In [4]:
data.shape

(46, 2)

In [5]:
p = []
np = []
for l in data.Class:
    if l == 'phishing':
        p.append(0)
        np.append(1)
    
#data.apply(lambda col: col.drop_duplicates().reset_index(drop=True))


In [6]:
data['Phishing']= p
data['Non-Phishing']= np
data.head()

,Class,Message,Phishing,Non-Phishing
0,phishing,From acteamco@server2.aking.com.my Sat Jan 24...,0,1
1,phishing,From rm@transservis.az Wed Jan 21 03:30:32 20...,0,1
2,phishing,From MAILER-DAEMON Thu Sep 28 09:57:25 2017\n\...,0,1
3,phishing,From m2266@cscv.qc.ca Fri Feb 6 08:48:37 2015...,0,1
4,phishing,From roberta@analiselaboratorios.com.br Mon M...,0,1


In [7]:
def removeHTMLtags(blob):
     soup = BeautifulSoup(blob,  "html.parser").text
     return soup





In [8]:
data['Message'] = data['Message'].apply(str)
data['Message_Clean'] = data['Message'].apply(lambda x
                                              : removeHTMLtags(x))


In [9]:
data.head(1)

,Class,Message,Phishing,Non-Phishing,Message_Clean
0,phishing,From acteamco@server2.aking.com.my Sat Jan 24...,0,1,From acteamco@server2.aking.com.my Sat Jan 24...


In [10]:
from nltk import word_tokenize, WordNetLemmatizer
tokens = [word_tokenize(sen) for sen in data.Message_Clean]


In [11]:

def lower_token(tokens): 
    return [w.lower() for w in tokens]    
    
lower_tokens = [lower_token(token) for token in tokens]

In [12]:
from nltk.corpus import stopwords
stoplist = stopwords.words('english')

In [13]:
def remove_stop_words(tokens): 
    return [word for word in tokens if word not in stoplist]

In [15]:
filtered_words = [remove_stop_words(sen) for sen in lower_tokens]
result = [' '.join(sen) for sen in filtered_words]

data['Email_Final'] = result
data['tokens'] = filtered_words


In [16]:
data = data[['Email_Final', 'tokens', 'Class', 'Phishing', 'Non-Phishing']]
data.head(1)


,Email_Final,tokens,Class,Phishing,Non-Phishing
0,acteamco @ server2.aking.com.my sat jan 24 03:...,"[acteamco, @, server2.aking.com.my, sat, jan, ...",phishing,0,1


In [17]:
data_train, data_test = train_test_split(data, 
                                         test_size=0.10, 
                                         random_state=42)

In [18]:
all_training_words = [word for tokens in data_train["tokens"] for word in tokens]
training_sentence_lengths = [len(tokens) for tokens in data_train["tokens"]]
TRAINING_VOCAB = sorted(list(set(all_training_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_training_words), len(TRAINING_VOCAB)))
print("Max sentence length is %s" % max(training_sentence_lengths))

23617 words total, with a vocabulary size of 2973
Max sentence length is 1142


In [19]:
all_test_words = [word for tokens in data_test["tokens"] for word in tokens]
test_sentence_lengths = [len(tokens) for tokens in data_test["tokens"]]
TEST_VOCAB = sorted(list(set(all_test_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_test_words), len(TEST_VOCAB)))
print("Max sentence length is %s" % max(test_sentence_lengths))

2147 words total, with a vocabulary size of 570
Max sentence length is 553


In [35]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1"
hub_layer = hub.KerasLayer(embedding, input_shape=[], 
                           dtype=tf.string, trainable=True)

In [36]:
MAX_SEQUENCE_LENGTH = 50
EMBEDDING_DIM = 300

In [57]:
tokenizer = Tokenizer(num_words=len(TRAINING_VOCAB), lower=True, char_level=False)

tokenizer.fit_on_texts(data_train['Email_Final'].tolist())

train_word_index = tokenizer.word_index
training_sequences = tokenizer.texts_to_sequences(data_train['Email_Final'].tolist())
maxlen = max([len(x) for x in training_sequences])
train_padded = pad_sequences(training_sequences, padding='post', truncating='post', maxlen=maxlen)
print("Word index:\n",train_word_index )

print("\nTraining sequences:\n", training_sequences)
print("\nPadded training sequences:\n", train_padded)
print("\nPadded training shape:", train_padded.shape)
print("Training sequences data type:", type(training_sequences))
print("Padded Training sequences data type:", type(train_padded))


Word index:
 {'com': 1, 'x': 2, '0': 3, 'hostedemail': 4, '2015': 5, "''": 6, 'org': 7, 'b': 8, 'monkey': 9, '10': 10, '5': 11, 'id': 12, 'jose': 13, '1': 14, 'content': 15, 'received': 16, 'none': 17, 'a': 18, 'account': 19, 'to': 20, 'jan': 21, 'net': 22, 'postfix': 23, 'dkim': 24, '19': 25, '8': 26, '0000': 27, 'message': 28, 'esmtp': 29, 'mime': 30, 'type': 31, 'utc': 32, 'neutral': 33, 'apple': 34, 'email': 35, 'status': 36, 'mail': 37, 'login': 38, 'delivered': 39, 'version': 40, 'forward': 41, 'suddenlink': 42, 'fri': 43, '16': 44, 'thu': 45, 'html': 46, 'information': 47, 'paypal': 48, 'rules': 49, 'transfer': 50, 'authentication': 51, 'results': 52, 'auth': 53, '248': 54, 'text': 55, 'encoding': 56, 'please': 57, '2': 58, 'charset': 59, 'mar': 60, '21': 61, '22': 62, 'spam': 63, 'click': 64, '36': 65, '0600': 66, '17': 67, 'size': 68, 'smtp': 69, 'header': 70, 'l': 71, 'return': 72, 'customer': 73, 'from': 74, 'date': 75, '12': 76, 'policy': 77, '3': 78, '64': 79, 'ro': 80, 'u

In [58]:
train_word_index = tokenizer.word_index

print("Found %s unique tokens." % len(train_word_index))
train_cnn_data = pad_sequences(training_sequences, 
                               maxlen=MAX_SEQUENCE_LENGTH)

Found 4205 unique tokens.


In [59]:
train_cnn_data = pad_sequences(training_sequences, maxlen=MAX_SEQUENCE_LENGTH)

SyntaxError: invalid syntax (<ipython-input-59-cae1cb4189ea>, line 2)

In [45]:
for word,index in train_word_index.items(): 
    train_embedding_weights[index,:] = hub_layer[word] if word in hub_layer else np.random.rand(EMBEDDING_DIM)
print(train_embedding_weights.shape)

TypeError: argument of type 'KerasLayer' is not iterable

In [44]:
test_sequences = tokenizer.texts_to_sequences(data_test["Email_Final"].tolist())
test_cnn_data = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [43]:
def ConvNet(embeddings, max_sequence_length, num_words, embedding_dim, labels_index):
 
    embedding_layer = Embedding(num_words,
                            embedding_dim,
                            weights=[embeddings],
                            input_length=max_sequence_length,
                            trainable=False)
    
    sequence_input = Input(shape=(max_sequence_length,), dtype='int32')
    embedded_sequences = embedding_layer(sequence_input)
    convs = []
    filter_sizes = [2,3,4,5,6]
    for filter_size in filter_sizes:
        l_conv = Conv1D(filters=200, 
                        kernel_size=filter_size, 
                        activation='relu')(embedded_sequences)
        l_pool = GlobalMaxPooling1D()(l_conv)
        convs.append(l_pool)
    l_merge = concatenate(convs, axis=1)
    x = Dropout(0.1)(l_merge)  
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.2)(x)
    preds = Dense(labels_index, activation='sigmoid')(x)
    model = Model(sequence_input, preds)
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['acc'])
    model.summary()
    return model

In [46]:
label_names = ['Phishing','Non-Phishing']

In [47]:
y_train = data_train[label_names].values

In [48]:
x_train = train_cnn_data
y_tr = y_train

In [49]:
model = ConvNet(training_embedding, 
                MAX_SEQUENCE_LENGTH, 
                len(train_word_index)+1, 
                EMBEDDING_DIM, 
                len(list(label_names)))

AttributeError: 'str' object has no attribute 'shape'

In [50]:
num_epochs = 3
batch_size = 34
print(type(x_train),type(y_train))

<class 'numpy.ndarray'> <class 'numpy.ndarray'>


In [51]:
x_train=np.asarray(x_train).astype(np.float32) 
le = LabelEncoder()
train_labels = le.fit_transform(y_train)

y_tr=np.asarray(y_tr).astype(np.float32) 

hist = model.fit(x_train, y_tr,
                 epochs=num_epochs,
                 validation_split=0.1,
                 shuffle=True, 
                 batch_size=batch_size)


ValueError: y should be a 1d array, got an array of shape (41, 2) instead.